In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# make a new directory for new images
!mkdir /content/drive/MyDrive/PyPSA_Africa_images/maxar/australia

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=00694d095e7885b5d60fb34b7224ae8d3c38eac0a7528eace97a739cc606aa6d
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import os
import wget

# move to respective dir
os.chdir('/content/drive/MyDrive/PyPSA_Africa_images/maxar/australia/')

In [ ]:
# define list of files to download
tifs = [
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-01-20/1040010036642800/1040010036642800.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-01-30/10300100788EF900/10300100788EF900.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-02-10/1030010077BD9E00/1030010077BD9E00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-05-03/103001007B036E00/103001007B036E00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-05-24/103001007D4FA400/103001007D4FA400.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-06-10/103001007F2EB200/103001007F2EB200.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-06-15/103001007FB4D600/103001007FB4D600.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-06-15/1030010080031400/1030010080031400.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-07-02/10300100800CD300/10300100800CD300.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-07-02/1030010080661B00/1030010080661B00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-07-11/10500100110EED00/10500100110EED00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-07-22/104001003F322700/104001003F322700.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-07-29/105001001159C100/105001001159C100.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-08-06/1030010083D81900/1030010083D81900.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-08-06/105001001178CB00/105001001178CB00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-08-09/104001003E7BE900/104001003E7BE900.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-08-10/1050010011843900/1050010011843900.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-08-11/1030010083CD5400/1030010083CD5400.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-08-11/1030010084841F00/1030010084841F00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-08-13/10500100118FC400/10500100118FC400.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-08-14/1030010084C36700/1030010084C36700.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-08-17/1050010011A2BD00/1050010011A2BD00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-09-10/1030010083422B00/1030010083422B00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-09-13/1030010084624100/1030010084624100.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-09-13/1030010086B86A00/1030010086B86A00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-09-16/1040010043B76800/1040010043B76800.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-09-18/1030010083895E00/1030010083895E00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-09-28/104001004322C700/104001004322C700.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-11-12/1030010088A8AF00/1030010088A8AF00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-11-12/10300100895DDE00/10300100895DDE00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-11-24/10400100444D6A00/10400100444D6A00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-12-04/1030010089174800/1030010089174800.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-12-09/103001008AD52700/103001008AD52700.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-12-09/103001008B82A700/103001008B82A700.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2018-12-26/1050010013AB4300/1050010013AB4300.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-01-30/103001008B406800/103001008B406800.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-02-07/1040010047337200/1040010047337200.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-02-27/1050010014BC7900/1050010014BC7900.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-03-28/105001001552E000/105001001552E000.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-04-08/103001008E256600/103001008E256600.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-04-09/1050010015899B00/1050010015899B00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-04-12/105001001596C100/105001001596C100.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-04-12/105001001596C200/105001001596C200.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-04-16/103001009042A400/103001009042A400.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-04-16/1050010015A3AE00/1050010015A3AE00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-04-30/105001001603FF00/105001001603FF00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-04-30/1050010016040000/1050010016040000.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-05-10/10300100901F3D00/10300100901F3D00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-05-10/103001009185A800/103001009185A800.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-05-29/1030010091BBEB00/1030010091BBEB00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-05-29/1030010094ADA800/1030010094ADA800.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-05-31/104001004C5D1100/104001004C5D1100.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-05-31/104001004CD39200/104001004CD39200.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-06-15/10300100921A2B00/10300100921A2B00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-06-15/1030010093328400/1030010093328400.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-07-01/104001004E982900/104001004E982900.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-07-13/1050010017610F00/1050010017610F00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-07-14/105001001769C500/105001001769C500.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-07-14/105001001769C600/105001001769C600.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-07-23/103001009793C100/103001009793C100.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-07-24/1050010017841E00/1050010017841E00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-02/103001009840FE00/103001009840FE00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-13/103001009646FC00/103001009646FC00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-13/10300100972B2C00/10300100972B2C00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-13/103001009795AC00/103001009795AC00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-13/1030010098AA7E00/1030010098AA7E00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-13/1030010099B36000/1030010099B36000.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-13/10500100180CAD00/10500100180CAD00.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-14/104001004ED5C200/104001004ED5C200.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-21/1040010051188200/1040010051188200.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-24/1050010018401100/1050010018401100.tif',
  'https://opendata.digitalglobe.com/events/australia-wildfires/pre-event/2019-08-27/10500100184E7D00/10500100184E7D00.tif',
  ]

In [10]:
from tqdm import tqdm

for tif in tqdm(tifs):
    wget.download(tif)

100%|██████████| 72/72 [1:00:50<00:00, 50.70s/it]
